In [152]:
%env VARIABLE=comptools-env

env: VARIABLE=comptools-env


In [153]:
import pandas as pd
books_data = pd.read_csv('data/books_data.csv') #, nrows=1000)
print(f"Books data has " + str(books_data.columns.size) + " columns with " + str(books_data.shape[0]) + " rows each.")
books_ratings = pd.read_csv('data/books_rating.csv') #, nrows=1000)
print(f"Books ratings has " + str(books_ratings.columns.size) + " columns with " + str(books_ratings.shape[0]) + " rows each.")

Books data has 10 columns with 212404 rows each.
Books ratings has 10 columns with 3000000 rows each.


In [154]:
# Titles
books_data_1 = books_data.drop(['image', 'previewLink', 'infoLink', 'publisher', 'ratingsCount'], axis=1, inplace=False)
books_ratings_1 = books_ratings.drop(['Id','Price', 'profileName', 'review/time'], axis=1, inplace=False)

books_data_1.rename(columns={'Title': 'title','publishedDate': 'published_date'}, inplace=True)
books_ratings_1.rename(columns={'Title': 'title','User_id':'user_id',  'review/score': 'score','review/helpfulness': 'helpfulness', 'review/text': 'text', 'review/summary': 'summary'}, inplace=True)
print(f"Books data now only has " + str(books_data_1.columns.size) + " columns - " + str(list(books_data_1.columns)))
print(f"Books ratings now only has " + str(books_ratings_1.columns.size) + " columns - " + str(list(books_ratings_1.columns))) 


Books data now only has 5 columns - ['title', 'description', 'authors', 'published_date', 'categories']
Books ratings now only has 6 columns - ['title', 'user_id', 'helpfulness', 'score', 'summary', 'text']


In [155]:
#Makes all titles lowercase; then removes any part of the title that includes a parenthesis, and replaces any ampersands with 'and'
books_data_1['title'] = books_data_1['title'].map(lambda s: s.lower() if type(s) == str else s)
books_data_1['title'] = books_data_1['title'].replace(r"\(.*\)","", regex=True)
books_data_1['title'] = books_data_1['title'].replace(r"\&","and", regex=True)

books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: s.lower() if type(s) == str else s)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"\(.*\)","", regex=True)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"\&","and", regex=True)

#Removes any characters that aren't letters or spaces.
books_data_1['title'] = books_data_1['title'].replace(r"[^a-zA-Z0-9\s]+", "", regex=True)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"[^a-zA-Z0-9\s]+", "", regex=True)

print("Both datasets' titles are now all lowercase and only include letters and spaces.")


Both datasets' titles are now all lowercase and only include letters and spaces.


In [156]:
# keep only unique titles in books_data_1
#size of books_data_1
prev_size = len(books_data_1['title'])
print(f"Books data has " + str(len(books_data_1['title'])) + " titles.")
books_data_1 = books_data_1.drop_duplicates(subset=['title'], keep='first')
print(f"Dropped " + str(prev_size - len(books_data_1['title'])) + " duplicate titles. Now there are only " + str(len(books_data_1['title'])) + " rows left. ")

#Remove NAN titles
print(f"" + str(books_ratings_1['title'].isnull().sum().sum()) + " total NAN values present in title")
books_ratings_1 = books_ratings_1.dropna(subset=['title'])
print(f"Now there are " + str(books_ratings_1['title'].isnull().sum().sum()))



Books data has 212404 titles.
Dropped 7500 duplicate titles. Now there are only 204904 rows left. 


In [157]:
#Drop duplicate reviews where same user reviews same book multiple times
print(f"Books ratings has " + str(len(books_ratings_1['title'])) + " rows.")
prevRatings = len(books_ratings_1['title'])
books_ratings_1.drop_duplicates(keep='first', inplace=True)
print(f"Dropped based on total duplicity. Books ratings now has " + str(len(books_ratings_1['title'])) + " rows. " + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")
prevRatings = len(books_ratings_1['title'])
books_ratings_1.drop_duplicates(subset=['user_id', 'title'], keep='first', inplace=True)
print(f"Dropped based on duplicate title. Books ratings now has " + str(len(books_ratings_1['title'])) + " rows. " + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")
prevRatings = len(books_ratings_1['title'])
books_ratings_1.drop_duplicates(subset=['user_id', 'text'], keep='first', inplace=True)
print(f"Dropped based on duplicate text. Books ratings now has " + str(len(books_ratings_1['title'])) + " rows. " + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")


Books ratings has 3000000 rows.
Dropped based on total duplicity. Books ratings now has 2432035 rows.  -567965 rows.
Dropped based on duplicate title. Books ratings now has 2027318 rows.  -404717 rows.
Dropped based on duplicate text. Books ratings now has 1748116 rows.  -279202 rows.


In [159]:
#Collect only ratings for books that have received at least 10 ratings.
#Add a new column in books_data_1 that counts the number of ratings for each book with a score of at least 10

#Note that by removing duplicate reviews first, counting the number of reviews for each book afterward will be more accurate.
print(f"Books_ratings has " + str(len(books_ratings_1['title'])) + " rows.")
prev_rows = len(books_ratings_1['title'])

n_ratings = books_ratings_1.groupby('title').count()['score']>=10
famous_books = n_ratings[n_ratings==True].index
books_ratings_1 = books_ratings_1[books_ratings_1['title'].isin(famous_books)]
n_ratings_1 = books_ratings_1['title'].value_counts().rename("n_ratings")
books_data_1 = books_data_1.merge(n_ratings_1, on='title', how='inner')

print(f"Books_ratings now has " + str(len(books_ratings_1['title'])) + " rows. -" + str(prev_rows - len(books_ratings_1['title'])) + " rows. ")
print(f"Books_data now has an additional column. (n_ratings)")


Books_ratings has 1748097 rows.
Books_ratings now has 1287419 rows. -460678 rows. 
Books_data now has an additional column. (n_ratings)


In [160]:
#purge books where n_ratings < 10 (redundant operation)
prevBooks = len(books_data_1['title'])
books_data_1 = books_data_1[books_data_1['n_ratings']>=10]
print(f"Books data has " + str(books_data_1.columns.size) + " columns with " + str(books_data_1.shape[0]) + " rows each." + " -" + str(prevBooks-len(books_data_1['title'])) + " rows.")

Books data has 6 columns with 33446 rows each. -0 rows.


In [161]:
#Remove ratings from users who have rated less than 20 books.
prevRatings = len(books_ratings_1['title'])
x = books_ratings_1.groupby('user_id').count()['score'] >= 20 # since this is what amazon requires before a user can get recommendations
considerable_users = x[x].index
books_ratings_1 = books_ratings_1[books_ratings_1['user_id'].isin(considerable_users)]

print(f"Books ratings has " + str(books_ratings_1.shape[0]) + " rows." + " -" + str(prevRatings-len(books_ratings_1['title'])) + " rows.")
print(f"" + str(len(books_ratings_1['user_id'].value_counts())) + " users remain who have rated at least 20 books.")

Books ratings has 6 columns with 168980 rows each. -1118439 rows.
3417 users remain who have rated at least 20 books.


In [162]:
avg_rating = books_ratings_1.groupby('title')["score"].mean().rename("avg_rating")
books_data_1 = books_data_1.merge(avg_rating, on='title', how='inner')
print(f"Books data now has an additional column. (avg_rating)")

Books data now has an additional column. (avg_rating)


In [163]:
#
def get_n_helpful(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        return int(denom)
    else:
        return x

def replace_fraction(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        if denom == '0':
            return 0
        else:
            return int(num) / int(denom)
    else:
        return x

books_ratings_1['helpfulness_count'] = books_ratings_1['helpfulness'].apply(get_n_helpful)
books_ratings_1['helpfulness_pct'] = books_ratings_1['helpfulness'].apply(replace_fraction)


In [164]:
import os
if not os.path.exists('clean_data'):
    os.makedirs('clean_data')

# save clean data
books_data_1.to_csv('clean_data/books_data_clean.csv', index=False)
books_ratings_1.to_csv('clean_data/books_rating_clean.csv', index=False)

prevBooks = len(books_data)
prevRatings = len(books_ratings)
newBooks = len(books_data_1)
newRatings = len(books_ratings_1)

print(f"First we had " + str(prevBooks) + " books and " + str(prevRatings) + " ratings.")
print(f"Now we have " + str(newBooks) + " books and " + str(newRatings) + " ratings. -" + str(round((prevBooks-newBooks)/prevBooks*100,2)) + "% books and -" + str(round((prevRatings-newRatings)/prevRatings*100,2)) + "% ratings.")

First we had 212404 books and 3000000 ratings.
Now we have 25402 books and 168980 ratings. -88.04% books and -94.37% ratings.


<h2>Code from here on down is not saved to the csv file.</h2>

In [165]:
books_data_1 = pd.read_csv('clean_data/books_data_clean.csv') #, nrows=1000)
print(f"Books data has " + str(books_data_1.columns.size) + " columns with " + str(books_data_1.shape[0]) + " rows each.")
books_ratings_1 = pd.read_csv('clean_data/books_rating_clean.csv') #, nrows=1000)
print(f"Books ratings has " + str(books_ratings_1.columns.size) + " columns with " + str(books_ratings_1.shape[0]) + " rows each.")

Books data has 7 columns with 25402 rows each.
Books ratings has 8 columns with 168980 rows each.


In [166]:
books_ratings_1['user_id'].value_counts()
#Looked at the ratings for the top 2 users and found nothing remarkable. The review text is cut off after a small number of words, so it is tough to see if they are automated reviews, as most of them more or less just describe the book rather than the reader's opinion of it.

user_id
AFVQZQ8PW0L       1963
A14OJS0VWMOSWO    1740
A1X8VZWTOG8IS6     765
A1K1JW1C5CUSUZ     677
A2F6N60Z96CAJI     557
                  ... 
ACK3TZEDR258N       20
AJN24TVEGZEI4       20
A1X9YMZQ2XXDCI      20
A31Y9DLKVASJQY      20
A2XZW1WV871DIO      20
Name: count, Length: 3417, dtype: int64

In [169]:
# extract all books rated by the top user by number of ratings, to check for duplicate reviews (spam)
books_ratings_1[books_ratings_1['user_id'] == 'AFVQZQ8PW0L']['summary'].value_counts()

summary
exciting thriller                          12
exciting romantic suspense                  8
deep character study                        6
A good read                                 6
exciting police procedural                  5
                                           ..
haunting classic war story                  1
fascinating historical mystery              1
engaging espionage romance                  1
exciting premise                            1
fantastic paranormal serial killer tale     1
Name: count, Length: 1779, dtype: int64